<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Order_66_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ### 5️⃣ Dependencias (Dale Clik a este triangulo)
!pip install rule34Py
!pip install python-resize-image
!pip install mplcursors
!pip install ipyplot
!pip install ipyvuetify
!pip install ipyevents
!pip install ipyevents
!pip install dash
!pip install dash_core_components
!pip install dash_html_componentsimport os
!apt-get install -y aria2

In [ ]:
#@markdown ### 5️⃣ Descargas
import os
import requests
from rule34Py import rule34Py
import shutil
import subprocess

# Crear una instancia de rule34Py
r34Py = rule34Py()

#@markdown Donde guardar las imagenes:
# Carpeta de destino para las imágenes
folder_path = '/content/images' #@param {type:"string"}
#@markdown Ej: /content/images Se guardara en la carpeta imagenes del entorno virtual, es decir no ocupara espacio (No hay necesidad de usar google drive)
#@markdown
#@markdown Ej: /content/drive/MyDrive/Dataset Se guardaran en tu drive, por lo que no tendras que descargarlas despues, pero si ocuparan espacio (Debes conetar tu drive si queres usarla)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
#@markdown
#@markdown
#@markdown Aca pones el tag que buscas , verificar que este bien escrito buscandolo en R34, si funciona en su buscador funciona en este tambien
Tag = 'tropius' #@param {type:"string"}
#@markdown Aca pones la cantidad de imagenes a descargar, maximo mil
Maximo = 132 #@param {type:"number"}
# Realizar la búsqueda de imágenes
search_results = r34Py.search([Tag], limit=Maximo)

# Descargar y guardar las imágenes utilizando requests
for result in search_results:
    image_url = result.image
    image_id = result.id

    if image_url:
        # Obtener el nombre del archivo y su extensión
        _, ext = os.path.splitext(image_url)

        # Verificar si el archivo es un gif o video
        if ext.lower() in ('.gif', '.webm', '.mp4'):
            print(f'El archivo es un gif o video. Saltando...: {image_url}')
            continue

        # Obtener la ruta completa de la imagen
        image_path = os.path.join(folder_path, f'{image_id}{ext}')

        # Descargar la imagen utilizando requests
        response = requests.get(image_url, stream=True)
        with open(image_path, 'wb') as file:
            shutil.copyfileobj(response.raw, file)

        print(f'Imagen guardada: {image_path}')
    else:
        print(f'URL de imagen vacía. Saltando...')


In [ ]:
#@markdown ### 5️⃣ Vizualizaor Low Data
import ipywidgets as widgets
from IPython.display import display
from PIL import Image

#@markdown Ruta de la carpeta donde se guardan las imágenes descargadas
folder_path = '/content/images' #@param {type:"string"}
# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

# Crear una lista de widgets de imagen para cada archivo
image_widgets = []
for file_name in files:
    image_path = os.path.join(folder_path, file_name)

    # Redimensionar la imagen si es necesario
    with Image.open(image_path) as img:
        if img.size[0] > 1200 or img.size[1] > 1200:
            img.thumbnail((1200, 1200), Image.ANTIALIAS)

    # Crear el widget de imagen y agregarlo a la lista
    image_widget = widgets.Image(value=open(image_path, 'rb').read())
    image_widgets.append(image_widget)

# Crear un contenedor de galería con VBox
gallery = widgets.VBox(children=image_widgets)

# Mostrar la galería
display(gallery)




In [ ]:
#@markdown ### 5️⃣ Visualizador big data
import cv2
import matplotlib.pyplot as plt
import os

#@markdown Ruta de la carpeta donde se guardan las imágenes descargadas
folder_path = '/content/images' #@param {type:"string"}

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

#@markdown Configurar el tamaño de la cuadrícula 10 X 10 = maximo 100 imagenes
num_rows = 10 #@param {type:"number"}
num_cols = 15 #@param {type:"number"}

# Crear una figura y una cuadrícula de subtramas
fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 20))

# Iterar sobre los archivos de imágenes y mostrarlos en las subtramas
for i, file in enumerate(files):
    # Leer la imagen usando OpenCV
    image_path = os.path.join(folder_path, file)
    img = cv2.imread(image_path)

    # Redimensionar la imagen si es necesario
    if img.shape[0] > 1200 or img.shape[1] > 1200:
        img = cv2.resize(img, (1200, 1200))

    # Mostrar la imagen en la subtrama correspondiente
    row = i // num_cols
    col = i % num_cols
    axs[row, col].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[row, col].axis('off')

    # Agregar una etiqueta de texto debajo de la imagen
    delete_label = axs[row, col].text(0.5, -0.1, 'Fbi Alert', transform=axs[row, col].transAxes,
                                      ha='center', va='top', color='red', fontsize=12)

    # Función para borrar la imagen al hacer clic en la etiqueta de texto
    def delete_image(event):
        if event.inaxes == axs[row, col]:
            # Eliminar la imagen del archivo
            os.remove(image_path)
            print(f"Imagen eliminada: {file}")
            # Actualizar la visualización para reflejar la eliminación
            axs[row, col].cla()
            axs[row, col].axis('off')
            axs[row, col].text(0.5, -0.1, 'Fbi Alert', transform=axs[row, col].transAxes,
                               ha='center', va='top', color='red', fontsize=12)
            plt.draw()

    # Conectar la función de eliminación al evento de clic en la etiqueta de texto
    delete_label.set_picker(True)
    fig.canvas.mpl_connect('pick_event', delete_image)

# Ajustar los espacios entre las subtramas
plt.tight_layout()

# Mostrar la figura
plt.show()



In [ ]:
#@markdown ### 5️⃣ Visualizador big data (me)

import dash
from dash import dcc, html
import base64
from PIL import Image
import io

#@markdown Ruta de la carpeta donde se guardan las imágenes descargadas
folder_path = '/content/images' #@param {type:"string"}

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

# Crear una lista para almacenar las imágenes
images = []

# Iterar sobre los archivos de imágenes y cargarlas
for file in files:
    # Cargar la imagen utilizando PIL
    image_path = os.path.join(folder_path, file)
    img = Image.open(image_path)

    # Convertir la imagen a formato base64
    img_data = io.BytesIO()
    img.save(img_data, format='PNG')
    img_base64 = base64.b64encode(img_data.getvalue()).decode('utf-8')

    # Agregar el formato base64 a la lista de imágenes
    images.append(img_base64)

# Crear la aplicación Dash
app = dash.Dash(__name__)

# Definir el diseño de la aplicación
app.layout = html.Div([
    html.H1("Cuadrícula de imágenes"),
    html.Div(id="image-grid", style={"display": "grid", "grid-template-columns": "repeat(10, 1fr)", "grid-gap": "10px"})
])

# Función para generar la cuadrícula de imágenes
def generate_image_grid():
    # Crear la lista de elementos HTML para mostrar las imágenes en una cuadrícula
    items = []
    for img_base64 in images:
        # Crear el elemento HTML para mostrar la imagen
        img_html = html.Img(src=f"data:image/png;base64,{img_base64}", width="200", height="200")
        items.append(img_html)

    return items

# Actualizar la cuadrícula de imágenes cuando se carga la página
@app.callback(
    dash.dependencies.Output("image-grid", "children"),
    dash.dependencies.Input("image-grid", "id")
)
def update_image_grid(id):
    return generate_image_grid()

# Ejecutar la aplicación
if __name__ == "__main__":
    app.run_server()



In [ ]:
#@markdown ### 5️⃣ Eiminador Big Data, magic code No Gpu
import os
import ipywidgets as widgets
from IPython.display import display
from PIL import Image

# Ruta de la carpeta donde se guardan las imágenes
folder_path = '/content/images'

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

# Crear una lista para almacenar las imágenes y sus contenedores
images = []
image_containers = []

# Cargar las imágenes y crear los contenedores correspondientes
for file in files:
    image_path = os.path.join(folder_path, file)
    image = Image.open(image_path)
    image.thumbnail((100, 100))
    image_widget = widgets.Image(value=image._repr_png_(), format='png', width=100, height=100)
    remove_button = widgets.Button(description='Eliminar', button_style='danger')
    image_container = widgets.VBox(children=[image_widget, remove_button], layout=widgets.Layout(align_items='center'))
    image_container.file = file
    images.append(image_container)
    image_containers.append(image_container)

# Función para eliminar una imagen
def remove_image(button):
    for i, image_container in enumerate(list(grid.children)):
        if image_container.children[1] == button:
            container = grid.children[i]
            grid.children = tuple(widget for widget in grid.children if widget != container)
            images.remove(image_container)
            # Eliminar la imagen del directorio
            image_file = os.path.join(folder_path, image_container.file)
            os.remove(image_file)
            break

# Crear una cuadrícula para mostrar las imágenes
grid = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(4, 250px)"))

# Agregar las imágenes y los botones a la cuadrícula
for image_container in image_containers:
    remove_button = image_container.children[1]
    remove_button.on_click(remove_image)
    grid.children += (image_container,)

# Mostrar la cuadrícula de imágenes
display(grid)



In [ ]:
#@markdown ### 5️⃣ Eiminador Big Data V2
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image

# Ruta de la carpeta donde se guardan las imágenes
folder_path = '/content/images' #@param {type:"string"}

# Obtener la lista de archivos en la carpeta
files = os.listdir(folder_path)

# Crear una lista para almacenar los nombres de las imágenes seleccionadas
selected_images = []

# Función para agregar o eliminar una imagen de la lista de seleccionadas
def toggle_image_selection(button):
    image_file = button.file
    if image_file in selected_images:
        selected_images.remove(image_file)
        button.button_style = ''
    else:
        selected_images.append(image_file)
        button.button_style = 'danger'

# Función para eliminar todas las imágenes seleccionadas
def remove_selected_images(button):
    deleted_images = []
    for image_file in selected_images:
        # Eliminar la imagen del directorio
        image_path = os.path.join(folder_path, image_file)
        os.remove(image_path)
        deleted_images.append(image_file)
    selected_images.clear()
    clear_output()
    print("Imágenes eliminadas: ", ", ".join(deleted_images))

# Crear una cuadrícula para mostrar las imágenes
grid = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(4, 250px)"))

# Agregar las imágenes y los botones a la cuadrícula
for file in files:
    image_path = os.path.join(folder_path, file)
    image = Image.open(image_path)
    largo = 150 #@param {type:"number"}
    alto = 150  #@param {type:"number"}
    image.thumbnail((largo + 10, alto + 10 ))
    image_widget = widgets.Image(value=image._repr_png_(), format='png', width= largo, height= alto)
    remove_button = widgets.Button(description='Seleccionar', button_style='')
    remove_button.file = file
    remove_button.on_click(toggle_image_selection)
    image_container = widgets.VBox(children=[image_widget, remove_button], layout=widgets.Layout(align_items='center'))
    grid.children += (image_container,)

# Botón para eliminar las imágenes seleccionadas
remove_selected_button = widgets.Button(description='Eliminar imágenes seleccionadas', button_style='')
remove_selected_button.on_click(remove_selected_images)

# Mostrar la cuadrícula de imágenes y el botón
display(grid, remove_selected_button)



In [13]:
#@markdown ### 5️⃣ Descarga de alta velocidad
import cv2
import os
import zipfile

#@markdown Ruta del directorio de imágenes
directorio_imagenes = '/content/images' #@param {type:"string"}

#@markdown Por sentido común se establece un tamaño máximo deseado para las imágenes de 1200 x 1200, en caso contrario se redimensionan al máximo
Altura = 1200 #@param {type:"number"}
Base = 1200 #@param {type:"number"}
tamano_maximo = (Base, Altura)

# Listas para almacenar las rutas de las imágenes redimensionadas y originales
imagenes_redimensionadas = []
imagenes_originales = []

# Recorre todos los archivos en el directorio de imágenes
for archivo in os.listdir(directorio_imagenes):
    ruta_archivo = os.path.join(directorio_imagenes, archivo)

    # Verifica si el archivo es una imagen
    if os.path.isfile(ruta_archivo) and archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
        imagen = cv2.imread(ruta_archivo)

        # Obtiene las dimensiones de la imagen
        altura, ancho = imagen.shape[:2]

        # Verifica si la imagen es mayor al tamaño máximo
        if altura > tamano_maximo[0] or ancho > tamano_maximo[1]:
            # Calcula las nuevas dimensiones proporcionales
            proporcion = min(tamano_maximo[0] / altura, tamano_maximo[1] / ancho)
            nueva_altura = int(altura * proporcion)
            nueva_ancho = int(ancho * proporcion)

            # Redimensiona la imagen
            imagen_redimensionada = cv2.resize(imagen, (nueva_ancho, nueva_altura))

            # Guarda la imagen redimensionada en una nueva ruta
            ruta_redimensionada = os.path.join(directorio_imagenes, f"redimensionada_{archivo}")
            cv2.imwrite(ruta_redimensionada, imagen_redimensionada)

            # Agrega la ruta de la imagen redimensionada a la lista correspondiente
            imagenes_redimensionadas.append(ruta_redimensionada)
        else:
            # Agrega la ruta de la imagen original a la lista correspondiente
            imagenes_originales.append(ruta_archivo)

#@markdown Crea un archivo zip y agrega las imágenes redimensionadas
archivo_zip = '/content/outputs.zip' #@param {type:"string"}
with zipfile.ZipFile(archivo_zip, 'w') as zip:
    for ruta_imagen in imagenes_redimensionadas:
        archivo_zip_nombre = os.path.basename(ruta_imagen)
        zip.write(ruta_imagen, archivo_zip_nombre)

# Descarga el archivo zip
from google.colab import files
files.download(archivo_zip)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@markdown ### 5️⃣ Descarga sin redimencionado
import os
import zipfile

#@markdown Ruta del directorio de imágenes Ruta del directorio de imágenes
directorio_imagenes = '/content/images' #@param {type:"string"}

# Crea un archivo zip y agrega las imágenes en el directorio
archivo_zip = '/content/outputs.zip'
with zipfile.ZipFile(archivo_zip, 'w') as zip:
    for archivo in os.listdir(directorio_imagenes):
        ruta_archivo = os.path.join(directorio_imagenes, archivo)

        # Verifica si el archivo es una imagen
        if os.path.isfile(ruta_archivo) and archivo.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
            zip.write(ruta_archivo, archivo)

# Descarga el archivo zip
from google.colab import files
files.download(archivo_zip)


In [ ]:
#@markdown ### 5️⃣ Elimina Todas los tags y imagenes de una carpeta para poder comenzar con otro prollecto.
import os

def delete_files(folder_path):
    # Verificar si la carpeta existe
    if not os.path.exists(folder_path):
        print("La carpeta no existe.")
        return

    # Obtener la lista de archivos en la carpeta
    files = os.listdir(folder_path)

    # Eliminar todos los archivos
    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Archivo eliminado: {file_path}")
        except Exception as e:
            print(f"No se pudo eliminar el archivo {file_path}. Error: {str(e)}")

# Ruta de la carpeta a borrar
folder_to_delete = '/content/images' #@param {type:"string"}

# Llamar a la función para borrar los archivos
delete_files(folder_to_delete)

In [ ]:
#@markdown Inicie el Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@markdown ### 5️⃣ Informacion de la funcion search
from rule34Py import rule34Py

# Crear una instancia de la clase rule34Py
r34Py = rule34Py()

# Obtener la ayuda de la función search
help(r34Py.search)


print('''Descripción:
Esta función se utiliza para buscar publicaciones en el sitio web de Rule34.

Argumentos:
- tags (list[str]): Lista de etiquetas de búsqueda.
- page_id (int, opcional): ID de la página de resultados.
- limit (int, opcional): Límite de publicaciones a obtener. Máximo 1000.
- deleted (bool, opcional): Indica si se incluyen publicaciones eliminadas.
- ignore_max_limit (bool, opcional): Indica si se ignora el límite máximo.

Valor de retorno:
- list: Lista de resultados de publicaciones. Si ocurre un error, la lista estará vacía.

Ayuda: Hoja de referencia rápida''')




In [ ]:
#@markdown ### 5️⃣ Guia de como usar los tags en R34
print(''' Ayuda: Hoja de referencia rápida para usar los tags
Búsqueda:

tag1 tag2: Busca publicaciones que tengan la etiqueta tag1 y tag2.
( tag1 ~ tag2 ): Busca publicaciones que tengan la etiqueta tag1 o tag2. Los paréntesis son importantes para agrupar las etiquetas entre las que se aplica el "o". Los espacios entre los paréntesis y las etiquetas también son importantes, ya que algunas etiquetas terminan en paréntesis.
night~: Búsqueda aproximada de la etiqueta night. Esto devolverá resultados como night, fight, bright, según la distancia de Levenshtein.
-tag1: Busca publicaciones que no tengan la etiqueta tag1.
ta*1: Busca publicaciones con etiquetas que comienzan con ta y terminan con 1.
user:bob: Busca publicaciones subidas por el usuario bob.
md5:foo: Busca publicaciones con el hash MD5 foo.
md5:foo*: Busca publicaciones cuyo hash MD5 comienza con foo.
rating:questionable: Busca publicaciones que tienen una calificación cuestionable.
-rating:questionable: Busca publicaciones que no tienen una calificación cuestionable.
parent:1234: Busca publicaciones que tienen 1234 como padre (e incluye la publicación 1234).
rating:questionable rating:safe: En general, combinar las mismas etiquetas de metadatos (las que tienen dos puntos) no funcionará.
rating:questionable parent:100: Sin embargo, se pueden combinar diferentes etiquetas de metadatos.
width:>=1000 height:>1000: Encuentra imágenes con un ancho mayor o igual a 1000 y una altura mayor a 1000.
score:>=10: Encuentra imágenes con una puntuación mayor o igual a 10. Este valor se actualiza una vez al día a las 12AM CST.
sort:updated:desc: Ordena las publicaciones por su orden de actualización más reciente.
''')

In [ ]:
#@markdown ### ⚔️ ¡El siguiente coddigo se usa para detectar imagenes similares  y eliminarlas! 🗡️


#@markdown Nuestra base de datos esta recontra contaminada con imagenes tan similares que no aportan nada al programa por lo que buscamos reucirlas, en general veras como tu set de datos pasa de 1000 imagenes a unas meras 50, y con  eso estamos bien.

#@markdown Cuanto mas alto sea el valor de similitud, menos probabilidades tendra de elimar imagenes no tan parecidas entre si, lo ideal es que limpie gran parte de la basura, lo recomendable ahora es hacer una copia de la carpeta con las imagenes de forma de poder verificar si las imagenes eliminadas fueron la corrrectas ( recorda que no buscamos ser muy especificos, no necistamos tanto)

#@markdown Cuando termines de eliminar las imagenes, rebiza el archivos de imagenes manualmente usando el siguiente programa: solo queremos usar su lector para ver si borramos lo corecto o debemos diminuir el similarity tresholder y borrar mas cosas. Tambien puedes borrar las que quedan manualmente usando la funcion de no recortar imagenes, pero recureda que la idea es hacerlo automaticamente: https://www.birme.net/?target_width=1000&target_height=900&no_resize=true&auto_focal=false


import os
import cv2
import imagehash
from PIL import Image
#@markdown Metodo de ImageHash - Investigacion relativa al estilo del eroge , inmediato
#@markdown
#@markdown 0 (No se eliminaran imagenes)
#@markdown
#@markdown 25: Elimina duplicadas de baja calidad - Valor recomendado para iniciar a eliminar
#@markdown
#@markdown 30: Comiensa a detectar los cambios de expreciones
#@markdown
#@markdown 40: Detectame correctamente imagenes similares con distintas expresiones
#@markdown
#@markdown 45: Comienzan a eliminarse las imagenes de cambio de ropa y poses identicas
#@markdown
#@markdown 55: Comienzan a eliminarse imagenes de alto valor
#@markdown
#@markdown 128 : Se eliminaran todas las imagenes

def calculate_ahash(image):
    # Cambiar el tamaño de la imagen a un tamaño fijo (8x8) para calcular el hash
    resized_image = cv2.resize(image, (8, 8))

    # Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Calcular el hash perceptual promedio (aHash)
    hash_value = imagehash.average_hash(Image.fromarray(gray_image))

    return hash_value

def normalize_difference(hash_diff):
    # Normalizar la diferencia de hashes a un rango de 0 a 128
    normalized_diff = 128 * (1 - hash_diff / 64)
    return normalized_diff

def remove_duplicates(folder_path, intensity_threshold):
    hashes = {}
    duplicates = []

    # Leer todas las imágenes en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)
            image_hash = calculate_ahash(image)

            # Comprobar si el hash ya existe
            is_duplicate = False
            for existing_hash in hashes:
                hash_diff = existing_hash - image_hash
                similarity = normalize_difference(hash_diff)
                if similarity <= intensity_threshold:
                    is_duplicate = True
                    duplicates.append(filename)
                    os.remove(image_path)
                    break

            if not is_duplicate:
                hashes[image_hash] = filename

    return duplicates

# Ruta de la carpeta que contiene las imágenes
NombreDelProllecto = "/content/images" #@param {type:"string"}
folder_path = f"/content/drive/MyDrive/Dataset/{NombreDelProllecto}"
intensity_threshold = 35  #@param {type:"slider", min:0, max:128, step:5}

# Eliminar las imágenes duplicadas con la intensidad especificada
duplicates = remove_duplicates(folder_path, intensity_threshold)

print(f"Se eliminaron {len(duplicates)} imágenes duplicadas.")


def count_images(folder_path):
    count = 0
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            count += 1
    return count

# Obtener la cantidad de imágenes que quedan en la carpeta
remaining_images = count_images(folder_path)

if remaining_images > 125:
    print("El programa tiene más de 125 imágenes, se recomienda aumentar un poco la intensidad.")
elif remaining_images < 40:
    print("El programa tiene menos de 40 imágenes, se recomienda la eliminación manual.")
else:
    print("El programa tiene una cantidad adecuada de imágenes.")

print(f"Quedan {remaining_images} imágenes en la carpeta.")